## Dijkstra revision

code exploring alternatives with Dijkstra algorithm

In [ ]:
#### import pandas as pd
import numpy as np
import geopandas as gpd
import networkx as nx
import osmnx as ox
from shapely.geometry import Point, LineString, Polygon
import matplotlib.pyplot as plt
from tqdm import tqdm
import multiprocessing as mp
import folium, itertools, os, time, warnings
from IPython.display import display, clear_output

warnings.filterwarnings("ignore")

### Directories

In [ ]:
os.getcwd()

In [ ]:
## Uncomment if filepath abovee is '/home/jovyan/work/RP-Kang2020/procedure/code'
os.chdir('../../') 

os.getcwd()

### Load Data

In [ ]:
# street networks
if not os.path.exists("data/raw/public/Chicago_Network.graphml"):
    G = ox.graph_from_place('Chicago', network_type='drive') # pulling the drive network the first time will take a while
    ox.save_graphml(G, 'raw/public/Chicago_Network.graphml')
else:
    G = ox.load_graphml('raw/public/Chicago_Network.graphml', node_type=str)
ox.plot_graph(G)

In [ ]:
hospitals = gpd.read_file('./data/raw/public/HospitalData/Chicago_Hospital_Info.shp')

### Process Hospitals + Street Networks

In [ ]:
def network_setting(network):
    _nodes_removed = len([n for (n, deg) in network.out_degree() if deg ==0])
    network.remove_nodes_from([n for (n, deg) in network.out_degree() if deg ==0])
    for component in list(nx.strongly_connected_components(network)):
        if len(component)<10:
            for node in component:
                _nodes_removed+=1
                network.remove_node(node)
    for u, v, k, data in tqdm(G.edges(data=True, keys=True),position=0):
        if 'maxspeed' in data.keys():
            speed_type = type(data['maxspeed'])
            if (speed_type==str):
                # add in try/except blocks to catch maxspeed formats that don't fit Kang et al's cases
                try:
                    if len(data['maxspeed'].split(','))==2:
                        data['maxspeed_fix']=float(data['maxspeed'].split(',')[0])                  
                    elif data['maxspeed']=='signals':
                        data['maxspeed_fix']=35.0 # drive speed setting as 35 miles
                    else:
                        data['maxspeed_fix']=float(data['maxspeed'].split()[0])
                        # print("Warning: the value of", data['maxspeed'], "is being converted to 35.0")   
                except:
                    data['maxspeed_fix']= 35.0 #miles
            else:
                try:
                    data['maxspeed_fix']=float(data['maxspeed'][0].split()[0])
                except:
                    data['maxspeed_fix']= 35.0 #miles
        else:
            data['maxspeed_fix']= 35.0 #miles
        # conversion confusion
        data['maxspeed_meters'] = data['maxspeed_fix']*26.8223 # convert (mile per hour) to (meters per minute)
        # meters / meters per minute === 
        data['time'] = float(data['length'])/ data['maxspeed_meters']
    print("Removed {} nodes ({:2.4f}%) from the OSMNX network".format(_nodes_removed, _nodes_removed/float(network.number_of_nodes())))
    print("Number of nodes: {}".format(network.number_of_nodes()))
    print("Number of edges: {}".format(network.number_of_edges()))
    return(network)

def hospital_setting(hospitals, G):
    hospitals['nearest_osm']=None
    for i in tqdm(hospitals.index, desc="Find the nearest osm from hospitals", position=0):
        hospitals['nearest_osm'][i] = ox.get_nearest_node(G, [hospitals['Y'][i], hospitals['X'][i]], method='euclidean') # find the nearest node from hospital location
    print ('hospital setting is done')
    return(hospitals)

In [ ]:
# clean street network
G = network_setting(G)

# find nearest node for each hospital
hospitals = hospital_setting(hospitals, G)

In [ ]:
distances = [10,20,30]
weights = [1,0, 0.68, 0.22]
distance_unit = 'time'
nearest_osm = hospitals['nearest_osm'][0]

### Current Functions (for reference)

In [ ]:
def dijkstra_cca(G, nearest_osm, distance, distance_unit = "time"):
    # creates subgraph (a list of nodes instead of writing a whole new network graph)
    road_network = G.subgraph(nx.single_source_dijkstra_path_length(G, nearest_osm, distance, distance_unit))  
    # creates an x and y point for each node
    nodes = [Point((data['x'], data['y'])) for node, data in road_network.nodes(data=True)]
    # constructs a multipart convex hull out of the subgraph of nodes
    polygon = gpd.GeoSeries(nodes).unary_union.convex_hull ## to create convex hull
    # turns the polygon into a geodatframme
    polygon = gpd.GeoDataFrame(gpd.GeoSeries(polygon)) ## change polygon to geopandas
    # renames geometry column
    polygon = polygon.rename(columns={0:'geometry'}).set_geometry('geometry')
    return polygon.copy(deep=True)

In [ ]:
def hospital_measure_acc (_thread_id, hospital, pop_data, distances, weights):
    ##distance weight = 1, 0.68, 0.22
    polygons = []
    for distance in distances:
        # append djikstra catchment calculation (uncomment to use)
        polygons.append(dijkstra_cca(G, hospital['nearest_osm'], distance))
        ## comment out original approach
        # polygons.append(calculate_catchment_area(G, hospital['nearest_osm'],distance))
    for i in reversed(range(1, len(distances))):
        polygons[i] = gpd.overlay(polygons[i], polygons[i-1], how="difference")
        
    num_pops = []
    for j in pop_data.index:
        point = pop_data['geometry'][j]
        for k in range(len(polygons)):
            if len(polygons[k]) > 0: # to exclude the weirdo (convex hull is not polygon)
                if (point.within(polygons[k].iloc[0]["geometry"])):
                    num_pops.append(pop_data['pop'][j]*weights[k])  
    total_pop = sum(num_pops)
    for i in range(len(distances)):
        polygons[i]['time']=distances[i]
        polygons[i]['total_pop']=total_pop
        polygons[i]['hospital_icu_beds'] = float(hospital['Adult ICU'])/polygons[i]['total_pop'] # proportion of # of beds over pops in 10 mins
        polygons[i]['hospital_vents'] = float(hospital['Total Vent'])/polygons[i]['total_pop'] # proportion of # of beds over pops in 10 mins
        polygons[i].crs = { 'init' : 'epsg:4326'}
        polygons[i] = polygons[i].to_crs({'init':'epsg:32616'})
    print('\rCatchment for hospital {:4.0f} complete'.format(_thread_id), end="")
    return(_thread_id, [ polygon.copy(deep=True) for polygon in polygons ]) 

### Djikstra revision using dictionary instead of subgraph

In [ ]:
# change the distance value doesn't seem to change the shortest path length
nx.single_source_dijkstra_path_length(G,
                                      hospitals['nearest_osm'][1], 
                                      cutoff=10,
                                      weight='time') # if we change this, the output values don't seem to change

In [ ]:
len(nx.single_source_dijkstra_path_length(G,
                                         hospitals['nearest_osm'][1], 
                                         cutoff=distances[0]))

In [ ]:
def dijkstra_cca_polygons(G, nearest_osm, distances, distance_unit = "time"):
    
    ## CREATE DICTIONARIES
    # create dictionary of nearest nodes
    nearest_nodes_30 = nx.single_source_dijkstra_path_length(G, nearest_osm, cutoff=distances[2]) # creating the largest graph from which 10 and 20 minute drive times can be extracted from
    
    # extract values within 20 minutes drive times
    nearest_nodes_20 = dict()
    for key, value in nearest_nodes_30.items():
        if value <= 20:
            nearest_nodes_20[key] = value
        
    # extract values within 10 minute drive times
    nearest_nodes_10 = dict()
    for key, value in nearest_nodes_30.items():
        if value <= 10:
            nearest_nodes_10[key] = value
            
    ## DERIVE SUBGRAPHS FROM DICTIONARY NODES
    # create service areas subgraphs for each
    service_area_30 = G.subgraph(nearest_nodes_30)
    service_area_20 = G.subgraph(nearest_nodes_20)
    service_area_10 = G.subgraph(nearest_nodes_10)

    ## TURN SUBGRAPHS INTO GDFS
    
    # create empty list to store polygons
    polygons = []
        
    # For 30 min drive time
    # creates an x and y point for each node
    nodes_30 = [Point((data['x'], data['y'])) for node, data in service_area_30.nodes(data=True)]
    # constructs a multipart convex hull out of the subgraph of nodes
    polygon_30 = gpd.GeoSeries(nodes_30).unary_union.convex_hull ## to create convex hull
    # turns the polygon into a geodatframme
    polygon_30 = gpd.GeoDataFrame(gpd.GeoSeries(polygon_30)) ## change polygon to geopandas
    # renames geometry column
    polygon_30 = polygon_30.rename(columns={0:'geometry'}).set_geometry('geometry')
    
    # For 20 min drive timee
    # creates an x and y point for each node
    nodes_20 = [Point((data['x'], data['y'])) for node, data in service_area_20.nodes(data=True)]
    # constructs a multipart convex hull out of the subgraph of nodes
    polygon_20 = gpd.GeoSeries(nodes_20).unary_union.convex_hull ## to create convex hull
    # turns the polygon into a geodatframme
    polygon_20 = gpd.GeoDataFrame(gpd.GeoSeries(polygon_20)) ## change polygon to geopandas
    # renames geometry column
    polygon_20 = polygon_20.rename(columns={0:'geometry'}).set_geometry('geometry')
    
    # For 10 min drive time
    # creates an x and y point for each node
    nodes_10 = [Point((data['x'], data['y'])) for node, data in service_area_10.nodes(data=True)]
    # constructs a multipart convex hull out of the subgraph of nodes
    polygon_10 = gpd.GeoSeries(nodes_10).unary_union.convex_hull ## to create convex hull
    # turns the polygon into a geodatframme
    polygon_10 = gpd.GeoDataFrame(gpd.GeoSeries(polygon_10)) ## change polygon to geopandas
    # renames geometry column
    polygon_10 = polygon_10.rename(columns={0:'geometry'}).set_geometry('geometry')
    
    # append each to list
    polygons.append(polygon_30)
    polygons.append(polygon_20)
    polygons.append(polygon_10)

    return polygons

In [ ]:
%time
dijkstra_cca_polygons(G, nearest_osm, distances)